### Getting and preprocessing the data
First we get the dataset online, then apply subsampling, then divide the dataset in equally long sentences

In [10]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
%%time
from collections import Counter
import numpy as np
import random
from itertools import dropwhile

def sampling(dataset,threshold=1e-4, min_count=5):
    
    # Count occurences of each word in the dataset 
    word_counts = Counter(dataset)  
    total_count = len(dataset)
    
    freqs = {word: count/total_count for word, count in word_counts.items()}
    p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
    #train_words = [word for word in dataset if random.random() < (1 - p_drop[word]) and word_counts[word]>min_count]
    #del dataset
    return p_drop

"Transforms a list of words to a list of sentences with length=len_sen"
def words_to_sentences(words):
    new_ds = []
    len_sen = int(len(words)/1700)
    len_sen = 20
    for i in range(0, len(words), len_sen):
        y = [words[i:i + len_sen]]
        new_ds.extend(y)
    return new_ds
    

CPU times: user 49.6 ms, sys: 26.1 ms, total: 75.7 ms
Wall time: 74.9 ms


### TEXT8 DATASET

In [2]:
from gensim.test.utils import datapath
import gensim.downloader as api
# Get dataset online
dataset = api.load('text8')
print('ntm')
# Convert to list of words
text8_ds = []
for x in dataset: 
    for y in x:
        text8_ds.append(y)
        
# Subsampling
#text8_ds = sampling(text8_ds)

# New dataset with sentences of length=20

sampling_dict_text8 = sampling(text8_ds)        

text8_ds_min_count = []
word_counts = Counter(text8_ds)
for x in text8_ds: 
    if ((word_counts[x] > 5) and (word_counts[x]<1407) ):
        text8_ds_min_count.append(x)

word_counts = Counter(text8_ds_min_count)
len(word_counts)
#text8_dataset = words_to_sentences(text8_ds)
text8_dataset = words_to_sentences(text8_ds_min_count)

ntm


In [15]:
wordsim_data = get_wordsim_data()
vocab = set(text8_ds_min_count)
l =[]
for x in wordsim_data: 
    if (x[0] in vocab) and (x[1] in vocab):
        l.append(0)
len(l)

140

In [11]:
wordsim_data = get_wordsim_data()
wordsim_data[0]
vocab = set(text8_ds_min_count)
len(vocab)

61103

In [13]:
import numpy as np
word_counts = dict(Counter(text8_ds))
word_counts_list = (list(word_counts.values()))
print(np.percentile(word_counts_list,75))
np.percentile(word_counts_list,99.5)


6.0


1407.469999999972

### Evaluation methods

In [3]:
from scipy import stats, spatial 
import csv, numpy as np
from scipy.stats.stats import pearsonr
from scipy import spatial 
#IMPORT DATA
def get_wordsim_data():
    wordsim_data = [] 
    with open('./data/wordsim/set1.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ',quotechar='|')
        for row in reader: 
            wordsim_data.append(row[0].split(',')[0:3])
    del wordsim_data[0]
    with open('./data/wordsim/set2.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ',quotechar='|')
        for i,row in enumerate(reader):
            if i!=0:
                wordsim_data.append(row[0].split(',')[0:3])

    wordsim_vocab = set()
    for x in wordsim_data:
        wordsim_vocab.add(x[0])
        wordsim_vocab.add(x[1])
    return wordsim_data

#len(wordsim_vocab.intersection(text8_dataset_first_sentence.vocab))
def wordsim_task(dict_emb):
    wordsim_data = get_wordsim_data()
    scores = []
    distances = []
    for task in wordsim_data: 
        if (task[0] in dict_emb.keys() ) and (task[1] in dict_emb.keys()):
            scores.append(float(task[2]))
            distances.append(spatial.distance.cosine(dict_emb[task[0]], dict_emb[task[1]]))
            
            
    #return stats.zscore(np.array([x[1] for x in out],dtype=float))
    return np.corrcoef(scores,distances)

#print(wordsim_task(gensim_emb))
#wordsim_task(dict_emb)

### Model

In [4]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import pdb

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cpu = torch.device('cpu')
gpu = torch.device("cuda:0")


class SkipGramModel(nn.Module):

    def __init__(self, vocab_size, emb_dimension):
        super(SkipGramModel, self).__init__()
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(vocab_size, emb_dimension, sparse=False)
        self.v_embeddings = nn.Embedding(vocab_size, emb_dimension, sparse=False)
        self.init_emb()
        

    def init_emb(self):
        initrange = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0,0)
        
            
    def forward(self, pos_u, pos_v,neg_v):
        pos_u = pos_u.view(-1).to(device)
        pos_v = pos_v.to(device)
        neg_v = neg_v.to(device)
        emb_u = self.u_embeddings(pos_u)
        samples = torch.cat([pos_v,Variable(neg_v)],1)
        emb_v = self.v_embeddings(samples)
        score = torch.bmm(emb_v, emb_u.unsqueeze(2)).squeeze()
        if(len(pos_u)==1):
            score[1:] = score[1:].neg()
        else:
            score[:,1:]=score[:,1:].neg()
        score = F.logsigmoid(score)
        return -1 * (torch.sum(score))/ pos_u.size(0)
    


In [5]:
from torch.utils.data import Dataset
from collections import defaultdict
import random

class wDataSet(Dataset):
    def __init__(self, dataset, sampling_dict, power=0.75,ctx_window=5):
        self.LEN_SEN =20
        self.sampling_dict=sampling_dict
        #assert( all(len(sentence)== self.LEN_SEN) for sentence in dataset)
        self.ctx_window = ctx_window
        self.dataset = dataset
        self.word2idx = dict()
        self.idx2word = dict()
        self.word_count = defaultdict(int)
        self.vocab_size = int()
        self.vocab = set()
        self.create_vocab()
        self.pairs = self.generate_pairs()
        self.key_pairs = self.generate_key_pairs(self.pairs)
        self.power = power        
        self.neg_table = self.make_neg_table(self.power)
        #self.len = self.__len__()
        self.build_pair_dict()
        
    
    def build_pair_dict(self):
        self.pair_dict = defaultdict(list)
        for x,y in self.key_pairs:
            self.pair_dict[x].append(y)
            

        
    def generate_pairs(self):
        print("Generating pairs")
        pairs = []
        for sentence in self.dataset:
            for i,word in enumerate(sentence):
                if(random.random() < 1- self.sampling_dict[word]):
                    for j in range(1,self.ctx_window+1):
                        if(i+j<len(sentence)):
                            pairs.append((word,sentence[i+j]))
                        if((i-j)>=0):
                            pairs.append((word,sentence[i-j]))
        return pairs
        
    def __len__(self):          
        len_dataset = len(self.dataset)     
        center_pairs = ((self.LEN_SEN - self.ctx_window*2)*self.ctx_window*2) 
        border_pairs = sum([self.ctx_window + i for i in range(self.ctx_window)])*2
        len_sen_without_last = (center_pairs + border_pairs)* (len_dataset-1)
        
        # The last sentence does not has the same length as the other ones, hence it's length needs to be computed otherwise
        len_last_sen = len(self.dataset[(len_dataset-1)])
        pairs_last_sen = 0
        for j in range(len_last_sen):
            if(j<self.ctx_window):
                # Checking if the rest of the sentence is smaller then the context window
                if(j+self.ctx_window >= len_last_sen):
                    diff = len_last_sen - 1- j 
                    pairs_last_sen += diff
                    pairs_last_sen += j
                else:
                    pairs_last_sen += (j+self.ctx_window)
            elif( j>= len_last_sen - self.ctx_window):
                pairs_last_sen += (len_last_sen-1-j+self.ctx_window)
            else:
                pairs_last_sen += (2*self.ctx_window)
    
        return len_sen_without_last + pairs_last_sen
        
        
   
    
    def get_neg_samples(self, count, batch_size):
        return torch.tensor(np.random.choice(list(self.idx2word.keys()),size=(batch_size)*count,replace=True,p=self.neg_table)).view(batch_size,-1)
   
    """ Defines the probability of choosing a negative sampling, set empiraccaly by mikolov"""
    def make_neg_table(self, power):
        pow_frequency = np.array([self.word_count[self.idx2word[i]] for i in range(len(self.word_count))])**power
        return pow_frequency / pow_frequency.sum()
        

    def generate_key_pairs(self,pairs):
        print("Generating key_pairs")
        key_pairs = []
        for x,y in pairs:
            key_pairs.append((self.word2idx.get(x),self.word2idx.get(y)))
        print("finished creating key_pairs")
        return key_pairs
    
    """"Creating vocabulary and creating dictionary with a one to one mapping int to word"""
    def create_vocab(self):
        print("Creating vocab")
        for i,sentence in enumerate(self.dataset):
            for word in sentence:
                self.word_count[word] += 1
                self.vocab.add(word)
        self.word2idx = {w: idx for (idx, w) in enumerate(self.vocab)}
        self.idx2word = {idx: w for (idx, w) in enumerate(self.vocab)}
        self.vocab_size = len(self.vocab)
        
             
    def __getitem__(self, idx):
        #Getting the number of pairs per sentence
        border_pairs = sum([self.ctx_window + i for i in range(self.ctx_window)])*2
        center_pairs = ((self.LEN_SEN - self.ctx_window*2)*self.ctx_window*2)
        n_pairs_in_sen = border_pairs + center_pairs
        id_sen = int(idx/n_pairs_in_sen)
        sen  = self.dataset[id_sen]
        pair_id_in_sen = idx - id_sen*(n_pairs_in_sen)
        counter = 0 
        for i,word in enumerate(sen):
            for j in range(1,self.ctx_window+1):
                if(i+j< len(sen)):
                    if(counter == pair_id_in_sen):
                        return(self.word2idx[word],self.word2idx[sen[i+j]])
                    counter+=1
                    
                if(i-j>=0):
                    if(counter == pair_id_in_sen):
                        return(self.word2idx[word],self.word2idx[sen[i-j]])
                    counter+=1
        

In [6]:
import torch
import pickle
from torch.utils.data import DataLoader
import copy
import time
import numbers
import pdb

class W2V():
    def __init__(self, data,dim=100, neg_samples=10, alpha=0.4, iterations=20, batch_size=2000, 
                 shuffle=True,use_cuda=True,workers=2,momentum=0,nesterov=False,step_size=1,gamma=1):
        self.momentum = momentum
        self.nesterov = nesterov
        self.step_size = step_size
        self.gamma = gamma
        self.shuffle = shuffle        
        self.batch_size = batch_size
        self.alpha = alpha
        self.dim = dim
        self.data = data
        self.workers = workers
        self.ctxw = self.data.ctx_window
        self.neg_samples = neg_samples
        self.use_cuda = use_cuda
        self.models = []
        self.optimizers = []
        self.ws_list = []
        self.loss_list = []
        self.model = SkipGramModel(len(self.data.vocab), self.dim)
        self.model.to(device)
    
        print(device)
        #self.optimizer = torch.optim.SGD(self.model.parameters(), lr=alpha, momentum=momentum,nesterov=nesterov)
        #self.scheduler = StepLR(self.optimizer, step_size=step_size, gamma=gamma)
        #self.optimizer = torch.optim.Adagrad(self.model.parameters(), lr=alpha)
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=alpha)


        self.iterations = iterations
 
    def train_with_loader(self,save_embedding=True):
        print('starting training')

        self.time=0
        best_score = -1
        prev_score = -1
        
        max_key = max(ds.pair_dict, key= lambda x: len(set(ds.pair_dict[x])))
        max_value = len(self.data.pair_dict[max_key])
        fifth = int(max_value/10)
        
        for epoch in range(self.iterations):

            percent = 0
            start = time.time()
            processed_batches = 0 
            pairs = 0
            cum_loss = 0 
            avg_loss =0
            best_loss = 10 
            

            for i in range(max_value):
                pos_v = []
                pos_u = []
                for x,y in ds.pair_dict.items():
                    if(len(y)>i):
                        pos_v.append(y[i])
                        pos_u.append(x)
              
                pos_v = torch.tensor(pos_v)
                pos_u = torch.tensor(pos_u)
                end = time.time()
                hours, rem = divmod(end-start, 3600)
                minutes, seconds = divmod(rem, 60)
                time_since_start = "Time:  {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)
                print("batch_size = " + str(len(pos_v)) + " || processed_batches = " + str(i) + "/" + str(max_value) + time_since_start,end ="\r")
                
                 
               
                    
                neg_v = self.data.get_neg_samples(self.neg_samples,pos_v.size()[0])
                pos_v = pos_v.view(len(neg_v),-1)
                self.optimizer.zero_grad()
                loss = self.model.forward(pos_u,pos_v,neg_v)
                cum_loss += loss
                loss.backward()
                self.optimizer.step()
                pairs += len(pos_u)
                processed_batches += 1
                
            print("\n{0:d} epoch of {1:d}".format(epoch+1, self.iterations))
            #avg_loss = cum_loss / processed_batches
            print(" {0:d} {1:d} batches, pairs {2:d}, cum loss: {3:.5f}".format(i,processed_batches, pairs,cum_loss))
            self.loss_list.append(cum_loss)
            self.time = time_since_start
            self.model = self.model.to(cpu)
            score = -1*(wordsim_task(self.get_embedding())[0][1])
            if(score < best_score):
                best_score = score
            print("Current score on wordsim Task: {}".format(score))
            self.ws_list.append(score)
            self.model = self.model.to(gpu)
            
        
            
            prev_score = score 
        
        if(save_embedding):
            self.save_embedding()
            

    def get_embedding(self):
        embedding_dict = dict()
        embedding = self.model.u_embeddings.weight.data.numpy()
        for i in range(len(self.data.idx2word)):
            embedding_dict[self.data.idx2word[i]]= embedding[i]
        return embedding_dict
    
    def save_embedding(self, with_loss=True):
        print('ntm')
        # Creating filename
        optim = "Optim" + str(self.optimizer).split(" ")[0] + "_"
        filename = "dict_emb_" +  optim + "_".join([x + str(y) for x,y in vars(self).items() if isinstance(y, numbers.Number)]) + ".pkl"
        
        # Getting Embedding
        self.model.to(torch.device('cpu'))
        dict_emb = w2v.get_embedding()
        
        # Adding loss history to embedding
        dict_emb['loss_list'] = [x.to(torch.device('cpu')) for x in self.loss_list]
        
        # Adding score list to embedding 
        dict_emb['ws_list'] = self.ws_list
        
                
        # Saving time spent to calculate 1 epoch
        dict_emb['time'] = self.time
        
        # Logging
        print("Saving embedding: {} to disk with ws_score: {} ".format(filename,dict_emb['ws_list']))
    
        # Writing embedding dictionnary to disk
        with open(filename, 'wb') as output:
            pickle.dump(dict_emb, output, pickle.HIGHEST_PROTOCOL)
        
        self.model.to(device)
        self.loss_list = [x.to(device) for x in self.loss_list]
    
 

            

In [7]:
# Snippet to test changes on very small dataset
ds = wDataSet(text8_dataset,sampling_dict_text8)


Creating vocab
Generating pairs
Generating key_pairs
finished creating key_pairs


In [ ]:
# w/o outliers advanced batching technique 
for x in [0.001,0.002,0.005,0.006,0.007]:
    print("learning rate = " + str(x))
    w2v = W2V(ds, neg_samples=10, alpha=x,iterations=1,shuffle=True)
    w2v.train_with_loader()


learning rate = 0.001
cuda:0
starting training
batch_size = 33 || processed_batches = 11652/11653Time:  00:08:01.643
1 epoch of 1
 11652 11653 batches, pairs 40968455, cum loss: 33617.76562
Current score on wordsim Task: 0.36541882514251367
ntm
Saving embedding: dict_emb_OptimAdam_momentum0_nesterovFalse_step_size1_gamma1_shuffleTrue_batch_size2000_alpha0.001_dim100_workers2_ctxw5_neg_samples10_use_cudaTrue_iterations1.pkl to disk with ws_score: [0.36541882514251367] 
learning rate = 0.002
cuda:0
starting training
batch_size = 33 || processed_batches = 11652/11653Time:  00:07:14.086
1 epoch of 1
 11652 11653 batches, pairs 40968455, cum loss: 32512.81836
Current score on wordsim Task: 0.5368784310166678
ntm
Saving embedding: dict_emb_OptimAdam_momentum0_nesterovFalse_step_size1_gamma1_shuffleTrue_batch_size2000_alpha0.002_dim100_workers2_ctxw5_neg_samples10_use_cudaTrue_iterations1.pkl to disk with ws_score: [0.5368784310166678] 
learning rate = 0.005
cuda:0
starting training
batch_siz

### Comparison to gensim

In [20]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec
gensim_emb = dict()
    
class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.cum_loss = 0
        self.loss_list = []
        self.ws_list = []
        self.prev_score = -1
        self.no_improvement =0


    def on_epoch_end(self, model):
        for word in vocab:
            gensim_emb[word] = model.wv[word]
            
        score = -1*wordsim_task(gensim_emb)[0][1]
        self.ws_list.append(score)
        
        if(score - self.prev_score < 0.0009):
            self.no_improvement +=1
            
        print("Epoch #{} end: cum_loss={}, ws_score={}".format(self.epoch,self.cum_loss,score))
        
        
        if(self.no_improvement == 2):
            print("No improvement in word similarity early stoppage")
            
            
            
        self.epoch += 1
        self.prev_score = score
    
    def on_batch_end(self, model):
        """Method called at the end of each batch.
        Parameters
        ----------
        model : :class:`~gensim.models.base_any2vec.BaseWordEmbeddingsModel`
            Current model.
        """
        self.cum_loss += model.get_latest_training_loss()
        



In [ ]:
epoch_logger = EpochLogger()
model = Word2Vec(epoch_logger = EpochLogger()
model = Word2Vec(text8_senteces,alpha= 0.05,size=100,window=5,negative=10,min_count=5, sample=10e4, iter=10, workers=1,sg=1, hs=0,callbacks=[epoch_logger],compute_loss=True),size=100,window=5,negative=10,min_count=5, sample=10e4, iter=10, workers=1,sg=1, hs=0,callbacks=[epoch_logger],compute_loss=True)